# Load Tensorflow and check GPU availability

In [1]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from core.loader import Loader
from models.model_wrapper import ModelWrapper

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-04-30 15:29:27.964303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-30 15:29:27.964366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-30 15:29:27.993625: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 15:29:28.057744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-30 15:29:29.190164: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-04-30 15:29:33.834267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-30 15:29:34.131901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-30 15:29:34.135391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Load input datasets

In [ ]:


# #############################################################
# EDIT this to specify benign / malicious datasets to use     #
# #############################################################
benign_dataset_filenames = [
    'parkets/benign/benign_2312_anonymized_HTML.parquet', 
    'parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames = [
    'parkets/phishing_2406_strict_HTML.parquet'
]

# lex-dga-830k-pick.parquet LABEL je jmeno rodiny
#  'parkets/malware_2406_strict_HTML.parquet'


# #############################################################
# EDIT this for to set appropriate labels (malware, dga, ...) #
# #############################################################
benign_label = "benign"
malicious_label = "phishing"
# #############################################################

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=1.0)
df = loader.load()

# dump columns names to text file


# use scaler to scale the data


In [3]:
with open('columns.txt', 'w') as f:
    for col in df.columns:
        f.write(f"{col}\n")

### Generate subsets of the training dataset

In [ ]:
from core.loader import Segmenter

# Define the aggregates that needs to be created

aggregates = [
    ["lex_"],
    ["lex_", "dns_", "ip_", "geo_"],
    ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
]

segmenter = Segmenter(df)
segmenter.create_base_subsets() # create base subsets
segmenter.create_aggregated_subsets(aggregates)
subset_dfs = segmenter.get_aggregated_subsets()

/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

### XGBOOST

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


model_wrapper = ModelWrapper(model_dir="models")
ARCH_NAME = "XgBoost"
VERSION = "v1.2"

# Dictionary to store XGBoost models and their performance metrics
xgb_models = {}
xgb_performance = {}

params = {        
    "objective": "binary:logistic",
    "tree_method": "gpu_hist",
    "sampling_method": "gradient_based",
    "max_depth": 12, 
    "eta": 0.09787878787878787, 
    "min_child_weight": 1, 
    "subsample": 0.595959595959596, 
    "alpha": 0, 
    "gamma": 0.06060606060606061, 
    "lambda": 2.0707070707070705, 
    "max_delta_step": 0, 
    "grow_policy": "depthwise",
    "max_bin": 512,
    "n_estimators": 600, 
    #"scale_pos_weight": scale_pos_weight,
    "random_state": 42  # Set the seed for each run
}

for prefix, subset_df in subset_dfs.items():
    print(f"\n🚀 Training XGBoost on '{prefix}' features...")
    
    X = subset_df.drop('label', axis=1)
    y = subset_df['label'].map(class_map)

    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42
    )

    xgb_model = XGBClassifier(**params)
    
    # Fit the model
    xgb_model.fit(X_train, y_train)
      
    y_pred = xgb_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    

    model_wrapper.save(xgb_model, arch_name=ARCH_NAME, label=malicious_label, prefix=f"{prefix}", version=VERSION)

    
    xgb_models[prefix] = xgb_model
    
    
    xgb_performance[prefix] = {
        'accuracy': acc,
        'classification_report': report,
    }
    
    print(f"✅ Accuracy for '{prefix}': {acc:.2f}")
    print(report)
    
        # shap analysis
    import shap
    import matplotlib.pyplot as plt
    
    # Create a SHAP explainer
    explainer = shap.Explainer(xgb_model, X_train)
    shap_values = explainer(X_test)
    shap.summary_plot(shap_values, X_test, feature_names=X.columns)
    plt.savefig(f"shap_summary_{ARCH_NAME}_{malicious_label}_{prefix}.png")


🚀 Training XGBoost on 'lex_agg' features...
Saving as stage: stage_1
✅ Accuracy for 'lex_agg': 0.97
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     81671
           1       0.96      0.83      0.89     16404

    accuracy                           0.97     98075
   macro avg       0.96      0.91      0.93     98075
weighted avg       0.96      0.97      0.96     98075


🚀 Training XGBoost on 'lex_+dns_+ip_+geo_agg' features...
Saving as stage: stage_2
✅ Accuracy for 'lex_+dns_+ip_+geo_agg': 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     81671
           1       0.98      0.95      0.96     16404

    accuracy                           0.99     98075
   macro avg       0.98      0.97      0.98     98075
weighted avg       0.99      0.99      0.99     98075


🚀 Training XGBoost on 'lex_+dns_+ip_+tls_+geo_+rdap_agg' features...
Saving as stage: stage_3
✅ Accuracy for 'lex_+

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

model_wrapper = ModelWrapper(model_dir="models")
 
# Dictionary to store LightGBM models and their performance metrics
lgb_models = {}
lgb_performance = {}

ARCH_NAME = "Lgbm"
VERSION = "v1.2"

# Iterate through each subset, train LightGBM, and evaluate it
for prefix, subset_df in subset_dfs.items():
    print(f"\n🚀 Training LightGBM on '{prefix}' features...")
    
    # Prepare data
    X = subset_df.drop('label', axis=1)
    y = subset_df['label']
    
    
    # map benign to 0 and malicious to 1
    y = y.map(class_map)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    # Initialize and train LightGBM with hyperparameter tuning
    lgb_model = LGBMClassifier(
        objective='binary', 
        metric='binary_logloss', 
        learning_rate=0.05, 
        num_leaves=31, 
        max_depth=-1, 
        n_estimators=250, 
        subsample=0.8, 
        colsample_bytree=0.8, 
        reg_alpha=0.1, 
        reg_lambda=0.1
    )
    
    lgb_model.fit(X_train, y_train)
    
    # save the model 
    model_wrapper.save(lgb_model, arch_name=ARCH_NAME, label=malicious_label, prefix=f"{prefix}", version=VERSION)
    
    # Store the model
    lgb_models[prefix] = lgb_model
    
    # Predict and evaluate the model
    y_pred = lgb_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    # Store performance
    lgb_performance[prefix] = {
        'accuracy': acc,
        'classification_report': report
    }
    
    # Print performance
    print(f"✅ LightGBM model accuracy for '{prefix}': {acc:.2f}")
    print(report)
    
    # shap analysis
    import shap
    import matplotlib.pyplot as plt
    
    # Create a SHAP explainer
    explainer = shap.Explainer(lgb_model, X_train)
    shap_values = explainer(X_test)
    shap.summary_plot(shap_values, X_test, feature_names=X.columns)
    plt.savefig(f"shap_summary_{ARCH_NAME}_{malicious_label}_{prefix}.png")


🚀 Training LightGBM on 'lex_agg' features...
[LightGBM] [Info] Number of positive: 148021, number of negative: 734653
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.327186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4012
[LightGBM] [Info] Number of data points in the train set: 882674, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.167696 -> initscore=-1.602044
[LightGBM] [Info] Start training from score -1.602044
Saving as stage: stage_1
✅ LightGBM model accuracy for 'lex_agg': 0.95
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     81671
           1       0.94      0.76      0.84     16404

    accuracy                           0.95     98075
   macro avg       0.95      0.88      0.91     98075
weighted avg       0.95      0.95      0.95     98075


🚀 Training LightGBM on 'lex_+dns_+ip_+geo_agg' features...